# Computer Vision Assignment 2
---

Semester: **Fall 2022**

Due date: **October 27th 2022, 11.59PM EST.**

## Instructions
---

You should perform this assignment using Google Colab. 
* Before starting, clone this assignment using `File > Save a copy in Drive`. 
* After you're done, go through the notebook and ensure that you have answered all questions.
* Finally, submit the ipynb `File > Download > Download .ipynb` on brightspace


## Introduction
---

This is a minimal notebook showing the basics of object detection using ~300 lines of code. In this assignment you will code an object detection network with functionality similiar to a Single Shot (or YOLO) detector.
 
For our purposes, we shall divide each image into rectangular crops called anchors. The job of our object detector is to classify each anchor as containing an object or not. If the anchor contains an object, we want to find out the change in the anchor position and size to best fit the object in it. Hence, object detection is a multi task network which does classification and regression.

In [ ]:
# load helper code
%cd /content
!rm -rf nyu-cv-object-detection
!git clone https://github.com/nikhilweee/nyu-cv-object-detection
%cd nyu-cv-object-detection

In [ ]:
import torch
import matplotlib
import torch.nn as nn
import torch.nn.functional as F

from utils.plot import visualize
from utils.encoder import ResNet18Encoder
from utils.dataset import get_dataloaders
from torchvision.ops import box_iou, nms
 
trainloader, valloader = get_dataloaders(batch_size=4)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

## Dataset
---

First, let's introduce our dataset. We shall work with an artificially generated dataset of different shapes (squares, triangles and circles) spread throughout an image. We have a Pytorch dataloader which outputs a `sample` and `target` pair. The `sample` contains the image and the `target` contains information about the bounding boxes. We have visualized the dataset below. You dont need to write any code here. However, feel free to dig into the functions that are called to better understand the structure of the dataset. 

In [ ]:
sample, target = iter(trainloader).next()
sample = torch.stack(sample, dim=0)

In [ ]:
visualize(sample)

In [ ]:
visualize(sample, target)

## Targets
---

The labels (or `target`s) for each `sample` in the datset have 2 kinds of information.

1. The bounding boxes which are a tensor of size (N x 4).  
Here is N is the number of objects in the image and 4 corresponds to:

  a. top left x coordinate  
  b. top left y coordinate  
  c. bottom right x coordinate  
  d. bottom right y coordinate  

2. The classification labels which are a tensor of size (N x 1).  
Here N corresponds to the number of objects in the image.  
The 1 corresponds to the label of the object (whether it is a triangle, square or circle).

For your convenience, here's what a `target` looks like.


In [ ]:
print(target[0])

## TODO 1 [20%]: Anchors
---

As described before, our object detection network heavily builds upon the concept of anchors. Each image is divided into equally spaced rectangles of different sizes. These rectangles are called anchors. Anchors are best explained through this image.

![Anchor Boxes](https://lilianweng.github.io/lil-log/assets/images/SSD-framework.png)

Our synthetic dataset has images of size 128x128. We shall divide the image into anchors of three different sizes - 40x40, 50x50 and 60x60. We shall use a stride length of 16, which is to say that two anchors of the same size will be separated by 16 pixels (horizontally and vertically). Keeping this in mind, we shall have 64 anchors (128/16 = 8 anchors horizontally and 128/16 = 8 anchors vertically) corresponding to each size. Since we are working with three different sizes, the image will be divided into 64x3 = 192 anchors.

Your first job is to write the code that creates these 192 anchors from an image of size 128x128. Each anchor will be represented by a 4-tuple (top left x, top left y, bottom right x, bottom right y). Therefore, the output of the function below should be a tensor of size `[192,4]`


In [ ]:
def get_anchors():
    """
    Generate 192 boxes where each box is represented by :
    [top_left_x, top_left_y, bottom_right_x, bottom_right_y]

    Each anchor position should generate 3 boxes according to the scales and ratios given.

    Return this result as a numpy array of size [192,4]
    """
    stride = 16
    image_size = 128
    scales = [40, 50, 60]
    
    anchors = []

    # TODO: Generate anchors

    return anchors

anchors = get_anchors()

assert anchors.size() == (192,4)

## TODO 2 [10%]: Model
---

Our object detection model will consist of two 'heads'. Both heads will process each of the 192 anchors that you generated in the previous section. The two heads are:

1. A **classification head** (`cls` layer in the picture) to detect whether the anchor has an object in it or not. This is done by measuring the IOU of the anchor with each of the bounding boxes from the `target`. If the IOU is > 0.7, we say that the chunk contains the object. If it is less than 0.3, we say the chunk only contains the background. If the value is somewhere in between, we say that it is a bad chunk and do not use it towards loss computation.

2. A **regression head** (`reg` layer in the picture) to calculate the change in height, width and the centre coordinates of the anchor to best fit the object. These offsets are 4 values for the height, width, center x and y coordinates.

A pictorial representation of the model is shown below. However, instead of `2k` in the given image we just have k scores for the `cls` layer. This is because we group triangles, squares and circles into one foreground class for this assignment. Of course, we can have seperate predictions for each class but that is added complexity and is not implemented here.

![Model Architecture](https://cdn-images-1.medium.com/max/1600/1*7heX-no7cdqllky-GwGBfQ.png)


Your next job is to construct the model. Most of it is already implemented, you just have to add the classification and regression heads.

In [ ]:
class ShapesModel(nn.Module):

    def __init__(self):
        super().__init__()

        # for each grid in the feature map we have 3 anchors of sizes: 40x40, 50x50, 60x60
        num_anchors = 3

        # regular resnet 18 encoder
        self.encoder = ResNet18Encoder()

        # a small conv net
        self.conv = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        # TODO: Add a convolutional Layer to predict the class. This is a head that predicts whether an anchor contains an object or not.
        self.cls = None

        # TODO: Add a convolutional Layer to predict the offsets. This is a regression head that calculates 4 offsets for each anchor.
        self.reg = None


    def permute_and_flatten(self, layer, N, A, C, H, W):
        # helper function that rearranges the input for the loss function

        layer = layer.view(N, -1, C, H, W)
        layer = layer.permute(0, 3, 4, 1, 2)
        layer = layer.reshape(N, -1, C)

        return layer


    def post_process(self, cls_pred, reg_pred):
        # helper function that gets outputs in the right shape for applying the loss

        N, AxC, H, W = cls_pred.size()
        Ax4 = reg_pred.size(1)

        A = Ax4 // 4
        C = AxC // A

        cls_pred = self.permute_and_flatten(
            cls_pred, N, A, C, H, W
        )

        reg_pred = self.permute_and_flatten(
            reg_pred, N, A, 4, H, W
        )

        return cls_pred.squeeze(), reg_pred.squeeze()


    def forward(self, x):

        x = self.encoder(x)

        # we take the 3rd output feature map of 
        # size 8 x 8 from the resnet18 encoder 

        x = x[3]

        x = F.relu(self.conv(x))

        cls_pred = self.cls(x)
        reg_pred = self.reg(x)

        cls_pred, reg_pred = self.post_process(cls_pred, reg_pred)

        return cls_pred, reg_pred


In [ ]:
model = ShapesModel()
sample, target = iter(trainloader).next()
sample = torch.stack(sample, dim=0)
cls_pred, reg_pred = model(sample)

assert cls_pred.size() == (4, 192) and reg_pred.size() == (4, 192, 4)

## TODO 3 [15%]: Preprocess
---

Now that we know how our model works, we need to prepare our dataset so that it can be used to train our model. Recall that our dataset provides bounding boxes and labels for each image. However, our model expects a class label and regression offsets for each anchor. In the following cells, you will implement `get_labels_and_offsets`, which calculates class labels and offsets for each anchor.

Offsets are calculated in case the anchor has an IOU > 0.7 with any real object. The offsets are calculated using `compute_offsets`, which has already been provided to you. You may also need to use [`torchvision.ops.box_iou`](https://pytorch.org/vision/main/generated/torchvision.ops.box_iou.html).

In [ ]:
def compute_offsets(targets, anchors):
    """
    This function returns the offsets that need to applied to anchors to morph them into targets.
    Both anchors and targets should be of the same shape, N x 4.
    The output would be offsets given in a torch tensor of size N x 4.
    """
    t_width = targets[:, 2] - targets[:, 0]
    t_height = targets[:, 3] - targets[:, 1]
    t_center_x = targets[:, 0] + 0.5 * t_width
    t_center_y = targets[:, 1] + 0.5 * t_height

    a_width = anchors[:, 2] - anchors[:, 0]
    a_height = anchors[:, 3] - anchors[:, 1]
    a_center_x = anchors[:, 0] + 0.5 * a_width
    a_center_y = anchors[:, 1] + 0.5 * a_height

    delta_x = (a_center_x - t_center_x) / t_width
    delta_y = (a_center_y - t_center_y) / t_height
    delta_scale_x = torch.log(a_width / t_width)
    delta_scale_y = torch.log(a_height / t_height)

    offsets = torch.cat(
        [
            delta_x.unsqueeze(0),
            delta_y.unsqueeze(0),
            delta_scale_x.unsqueeze(0),
            delta_scale_y.unsqueeze(0),
        ],
        dim=0,
    )
    return offsets.permute(1, 0)

In [ ]:
def get_labels_and_offsets(targets, anchors):
    '''
    INPUT:
    targets: [N x 4]: Bounding boxes in the image.
    anchors: [192 x 4]: Anchor boxes of an image. 

    OUTPUT: 
    labels: [192 x 1]: Class labels for each anchor. 1 is for foreground, 0 is for background and -1 is for a bad anchor.
    offsets: [192 x 4]: Offsets for anchor to best fit the bounding box object. Calculated if class label is 1.

    '''
    high_threshold = 0.7
    low_threshold = 0.3

    N = anchors.size(0)
    
    # TODO: Calculate labels and offsets

    return labels.squeeze(), offsets.squeeze()

In [ ]:
def get_batch(sample, target):
    '''
    Input
    target => Set of bounding boxes for each image.
    Sample => Each image
    Output:
    Bounding box offsets and class labels for each anchor.
    '''

    all_labels = []
    all_offsets = []
    for s, t in zip(sample, target):
        targets = t['bounding_box'].to(device)
        labels, offsets = get_labels_and_offsets(targets, anchors)
        all_labels.append(labels)
        all_offsets.append(offsets)
    
    all_labels = torch.stack(all_labels, dim=0)
    all_offsets = torch.stack(all_offsets, dim=0)

    return all_labels, all_offsets

In [ ]:
sample, target = iter(trainloader).next()
sample = torch.stack(sample, dim=0)

labels, offsets = get_batch(sample, target)
assert labels.size() == (4, 192) and offsets.size() == (4, 192, 4)

This is the meat of object detection right there ! A major part of the exercise is the correct calculation of the anchors for an image and its ground truth. Now that this is done, we can move on to our neural network training code.

## TODO 4 [20%]: Loss
---

The two heads in our model will use different losses.

1. The classification head shall use the binary cross entropy loss. The loss is calculated only for those anchors which have been classified as foreground or background (IOU > 0.7 or IOU < 0.3) and should not be calculated for anchors with (0.3 < IOU < 0.7).

Since there are too many background anchors, the model can easily predict `background` for each anchor and still get a high accuracy. To prevent this, we may sample the foreground more frequently (say 3x more than the background). This strategy of sampling less frequent examples more often is called negative sampling. Since we're dealing with relatively small number of anchors (192), this problem shouldn't affect us yet. However, if we were to use more anchors, we would have to be more careful.

2. For the regression head, use a Smooth L1 loss. This works well and prevents outliers. Only calculate this loss when there is a valid offset.

Your next task is to write the code for the two losses. 

**Extra Credit**: Implement negative sampling as described above.


In [ ]:
def label_loss(pred_label, gt_label):
    # TODO return label loss

def offset_loss(pred_offset, gt_offset, gt_label):
    # TODO return offset loss 


## TODO 5 [10%]: Training 
---

This is the training function. It is complete in itself, but you're free to change this code to suit your needs. Make sure that your model is able to train well without any hiccups. In particular, we're looking for decreasing loss values without any NaNs.

In [ ]:
def train(epoch, model, trainloader, optimizer):
        total_loss = 0
        running_offset_loss = 0
        running_label_loss = 0
        for i, (images, targets) in enumerate(trainloader):
            images = torch.stack(images, dim=0).to(device)
            gt_labels, gt_offsets = get_batch(images, targets)
            pred_labels, pred_offsets = model(images)
            loss_label = label_loss(pred_labels, gt_labels)
            loss_offset = offset_loss(pred_offsets, gt_offsets, gt_labels)
            loss = loss_label + loss_offset

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_label_loss += loss_label.item()
            running_offset_loss += loss_offset.item()

            if (i+1) % 500 == 0:
                print(f'Iter {i+1} | Class Loss: {running_label_loss / (i+1)} Offset Loss: {running_offset_loss / (i+1)}')
            
        avg_running_label_loss = float(running_label_loss / len(trainloader))
        avg_running_offset_loss = float(running_offset_loss / len(trainloader))
        print(f'Epoch: {epoch+1} | Class Loss: {avg_running_label_loss}, Offset Loss: {avg_running_offset_loss} \n')

In [ ]:
# TODO: Train your model

model = ShapesModel()
model.to(device)
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1, momentum=0.9, weight_decay=1e-4
)
for epoch in range(5):
    train(epoch, model, trainloader, optimizer)

## Validation
---

Now that we have a trained model, let's move on to visualizing some results. First, let's have a look at the validation data.

In [ ]:
sample, target = iter(valloader).next()
sample = torch.stack(sample,dim=0)

In [ ]:
visualize(sample, target)


## Non-Maximal Supression
---
To visualize predictions from our model, we perform NMS on the final predictions to clear up the output. This is done as a lot of anchors near to the actual object will fire resulting in a lot of boxes. NMS handles these multiple predictions by merging and giving a single box for a predicted object instead of multiple. This is best shown through the following image.

![NMS](https://miro.medium.com/max/1000/0*WI5_K3bAbYaRyzE-.png)

## TODO 6 [25%]: Visualize
---

Now onto the final stretch! Your final task is to write code to visualize the model predictions on an image from the validation set. 


You will implement the `visualize_preds` function. This function will perform three subtasks.  
  (a). Select what anchors to draw on the image. Generally, if the foreground probability is > 0.7, it's a good idea to use the anchor.  
  (b). Adjust the anchors using the offsets. For this, you will need to _invert_ the `compute_offsets` function described above.  
Specifically, you will write the function `apply_offsets` that is related to `compute_offsets` in the following manner.
```
targets = apply_offsets(anchors, compute_offsets(targets, anchors))
```
  (c). Finally, you will use NMS to filter out anchors. You can use [`torchvision.ops.nms`](https://pytorch.org/vision/stable/generated/torchvision.ops.nms.html)

In [ ]:
def apply_offsets(anchors, offsets):
    """
    This function appiles offsets to anchors and returns the result.
    Both anchors and offsets should be of the same shape, N x 4.
    offsets is the result from compute_offsets function.
    """
    # TODO: apply offsets to anchors


In [ ]:
def visualize_preds(model, sample):
    model = model.to('cpu')

    label_preds, offset_preds = model(sample)

    # TODO: select foreground anchors, apply offsets and use NMS

    visualize(sample, predictions)

visualize_preds(model, sample)

## Conclusion
---
Not bad, right ? This is a very barebones implementation and trained for a very short amount of epochs. However this is the gist of most anchor based state of the art detectors out today. We can improve accuracy by doing multiscale training and incorporating a Feature Pyramid Network amongst other things, but that is for you to discover :)

## References
---

Some foundational/good papers on object detection. This list is non exhaustive

1. [Single Shot Detector](https://arxiv.org/abs/1512.02325)
2. [YOLO](https://arxiv.org/abs/1612.08242)
3. [Faster RCNN](https://arxiv.org/abs/1506.01497)
4. [Feature Pyramid Network](https://arxiv.org/abs/1612.03144)
5. Bonus Read: [DETR- State of the art Object detector based on Transformers](https://arxiv.org/abs/2005.12872)
